# Imports

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import pandas as pd

# Getting raw data

In [3]:
fbref = pd.read_csv("data/logs_all.csv")

In [4]:
# fbref

In [5]:
# fbref.iloc[0]

In [6]:
fpl = pd.read_csv("data/fpl_players.csv")

In [7]:
# fpl[(fpl["name"]=="Brenden Aaronson") & (fpl["season"]=="2022-23")]

In [8]:
# removing rows with GW == NaN
fpl = fpl[fpl['GW'].notna()]

In [9]:
fpl[(fpl["season"] == "2022-23") & (fpl["name"] == "Ben Davies") & (fpl["GW"] == 2)]

,Unnamed: 0,index,season,name,position,team,assists,bonus,bps,clean_sheets,...,value,was_home,yellow_cards,GW,xP,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,starts
49676,49676,746,2022-23,Ben Davies,DEF,Liverpool,0,0,0,0,...,40,True,0,2,0.0,0.0,0.0,0.0,0.0,0.0
49910,49910,980,2022-23,Ben Davies,DEF,Spurs,1,0,20,0,...,50,False,0,2,3.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# sorted(fpl["team"].unique())

In [11]:
# sorted(fbref["Squad"].unique())

# Merging

In [12]:
def team_fbref_to_fpl(team_name):
    if team_name == "Nott'ham Forest":
        return "Nott'm Forest"
    if team_name == "Leicester City":
        return "Leicester"
    if team_name == "Leeds United":
        return "Leeds"
    if team_name == "Manchester City":
        return "Man City"
    if team_name == "Manchester Utd":
        return "Man Utd"
    if team_name == "Tottenham":
        return "Spurs"
    if team_name == "Newcastle Utd":
        return "Newcastle"
    
    return team_name

In [13]:
def neutralize_name(name):
    return name.replace(" ", "-").replace("ï", "i").replace("é", "e").replace("á", "a").replace("ó", "o").replace("Á", "A").replace("ø", "o").replace("ć", "c").replace("í", "i").replace("ú", "u").replace("Ć", "C").replace("ã", "a").replace("ğ", "g").replace("ş", "s").replace("É", "E").replace("ñ", "n").replace("Ł", "L").replace("ń", "n").replace("ß", "ss").replace("ç", "c").replace("İ", "I").replace("č", "c").replace("ö", "o").replace("ë", "e").replace("š", "s").replace("ä", "a").replace("Ç", "C").replace("ü", "u").replace("Ø", "O").replace("ú", "u").replace("ú", "u").replace("î", "i")

In [14]:
from src.match_names import name_fbref_to_fpl

In [15]:
# add neutral name columns
fpl["neutral_name"] = fpl["name"].apply(neutralize_name)
fbref["neutral_name"] = fbref["Name"].apply(neutralize_name)
fbref["FPL_name"] = fbref["neutral_name"].apply(name_fbref_to_fpl)
fbref["FPL_team"] = fbref["Squad"].apply(team_fbref_to_fpl)
fbref["FPL_opp_team"] = fbref["Opponent"].apply(team_fbref_to_fpl)

In [16]:
# fpl.info()

In [17]:
fbref.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24683 entries, 0 to 24682
Data columns (total 58 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    24683 non-null  int64  
 1   Date          24683 non-null  object 
 2   Day           24683 non-null  object 
 3   Comp          24683 non-null  object 
 4   Round         24683 non-null  object 
 5   Venue         24683 non-null  object 
 6   Squad         24683 non-null  object 
 7   Opponent      24683 non-null  object 
 8   Start         24683 non-null  object 
 9   Pos           24640 non-null  object 
 10  Min           24680 non-null  float64
 11  Gls           24683 non-null  float64
 12  Ast           24683 non-null  float64
 13  PK            24683 non-null  float64
 14  PKatt         24683 non-null  float64
 15  Sh            24583 non-null  float64
 16  SoT           24583 non-null  float64
 17  CrdY          24683 non-null  float64
 18  CrdR          24683 non-nu

In [18]:
join_columns_fbref = ['FPL_name', 'Season', 'FPL_team', 'Was Home', 'FPL_opp_team']
join_columns_fpl = ['neutral_name', 'season', 'team', 'was_home', 'opp_team_name']

In [19]:
result = fbref.merge(fpl, left_on=join_columns_fbref, right_on=join_columns_fpl, how='left').drop('Unnamed: 0_x', axis = 1)

In [20]:
result

,Date,Day,Comp,Round,Venue,Squad,Opponent,Start,Pos,Min,...,was_home,yellow_cards,GW_y,xP,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,starts,neutral_name_y
0,2021-08-15,Sun,Premier League,Matchweek 1,Away,West Ham,Newcastle Utd,Y,LB,90.0,...,False,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,Aaron-Cresswell
1,2021-08-23,Mon,Premier League,Matchweek 2,Home,West Ham,Leicester City,Y,LB,90.0,...,True,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,Aaron-Cresswell
2,2021-08-28,Sat,Premier League,Matchweek 3,Home,West Ham,Crystal Palace,Y,LB,90.0,...,True,0.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,Aaron-Cresswell
3,2021-09-11,Sat,Premier League,Matchweek 4,Away,West Ham,Southampton,Y,LB,90.0,...,False,0.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,Aaron-Cresswell
4,2021-09-19,Sun,Premier League,Matchweek 5,Home,West Ham,Manchester Utd,Y,LB,90.0,...,True,0.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,Aaron-Cresswell
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24678,2023-02-05,Sun,Premier League,Matchweek 22,Home,Tottenham,Manchester City,N,"CM,AM",7.0,...,True,0.0,22.0,0.5,0.00,0.00,0.00,0.27,0.0,Yves-Bissouma
24679,2023-05-06,Sat,Premier League,Matchweek 35,Home,Tottenham,Crystal Palace,N,0,0.0,...,True,0.0,35.0,0.0,0.00,0.00,0.00,0.00,0.0,Yves-Bissouma
24680,2023-05-13,Sat,Premier League,Matchweek 36,Away,Tottenham,Aston Villa,N,CM,30.0,...,False,0.0,36.0,0.0,0.01,0.01,0.00,0.08,0.0,Yves-Bissouma
24681,2023-05-20,Sat,Premier League,Matchweek 37,Home,Tottenham,Brentford,Y,CM,90.0,...,True,1.0,37.0,0.5,0.04,0.04,0.00,1.29,1.0,Yves-Bissouma


In [24]:
# result.columns.to_list()

# Saving merged df to file

In [22]:
# fbref[["Name", "GW", "Season", "Squad", "Total Points"]].to_csv("data/fpl_fbref_players.csv")
result.to_csv("data/fpl_fbref_players2.csv")

In [23]:
result["Name"].unique()

array(['Aaron-Cresswell', 'Aaron-Hickey', 'Aaron-Ramsdale',
       'Aaron-Wan-Bissaka', 'Abdoulaye-Doucoure', 'Adam-Armstrong',
       'Adam-Forshaw', 'Adam-Lallana', 'Adam-Smith', 'Adam-Webster',
       'Adama-Traore', 'Ainsley-Maitland-Niles', 'Albert-Sambi-Lokonga',
       'Alejandro-Garnacho', 'Aleksandar-Mitrovic', 'Alex-Iwobi',
       'Alex-McCarthy', 'Alex-Moreno', 'Alex-Oxlade-Chamberlain',
       'Alexander-Isak', 'Alexander-Mighten', 'Alexis-Mac-Allister',
       'Alisson', 'Allan-Saint-Maximin', 'Alphonse-Areola',
       'Amadou-Onana', 'Andre-Ayew', 'Andreas-Pereira', 'Andrew-Moran',
       'Andrew-Robertson', 'Angelo-Ogbonna', 'Anthony-Elanga',
       'Anthony-Gordon', 'Anthony-Martial', 'Antoine-Semenyo',
       'Antonee-Robinson', 'Antony', 'Armando-Broja',
       'Armel-Bella-Kotchap', 'Arnaut-Danjuma', 'Ashley-Young',
       'Asmir-Begovic', 'Aymeric-Laporte', 'Ayoze-Perez', 'Ben-Chilwell',
       'Ben-Davies', 'Ben-Doak', 'Ben-Godfrey', 'Ben-Johnson', 'Ben-Mee',
     